In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

You should consider upgrading via the 'c:\users\narayan jha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\narayan jha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

In [4]:
spark=SparkSession.builder.appName('final_project').getOrCreate()

In [5]:
!curl -O https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
 54 60682   54 32768    0     0  14290      0  0:00:04  0:00:02  0:00:02 14321
100 60682  100 60682    0     0  23507      0  0:00:02  0:00:02 --:--:-- 23547


In [6]:
df=spark.read.csv('NASA_airfoil_noise_raw.csv', header=True, inferSchema=True)
df.show()

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
|     2500|          0.0|     0.3048|              71.3|             0.00266337|   125.571|
|     3150|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     4000|          0.0|     0.3048|              71.3|             0.00266337|

In [7]:
rowcount1=df.count()
rowcount1

1522

In [10]:
df = df.dropDuplicates()
rowcount2=df.count()
rowcount2

1503

In [11]:
df = df.dropna()
rowcount3=df.count()
rowcount3

1499

In [12]:
df = df.withColumnRenamed('SoundLevel','SoundLevelDecibels')
df.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|           115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|           121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|           115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|           131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|           131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|           122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|           129.116|


In [13]:
df.write.mode("overwrite").parquet("NASA_airfoil_noise_cleaned.parquet")

In [14]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("New column name = ", df.columns[-1])

import os

print("NASA_airfoil_noise_cleaned.parquet exists :", os.path.isdir("NASA_airfoil_noise_cleaned.parquet"))

Part 1 - Evaluation
Total rows =  1522
Total rows after dropping duplicate rows =  1503
Total rows after dropping duplicate rows and rows with null values =  1499
New column name =  SoundLevelDecibels
NASA_airfoil_noise_cleaned.parquet exists : True


In [15]:
df = spark.read.parquet("NASA_airfoil_noise_cleaned.parquet")
df

DataFrame[Frequency: int, AngleOfAttack: double, ChordLength: double, FreeStreamVelocity: double, SuctionSideDisplacement: double, SoundLevelDecibels: double]

In [16]:
rowcount4=df.count()
rowcount4

1499

In [18]:
assembler = VectorAssembler(inputCols=['Frequency','AngleOfAttack','ChordLength','FreeStreamVelocity','SuctionSideDisplacement'], outputCol='features')

In [22]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [28]:
lr=LinearRegression(featuresCol='scaledFeatures', labelCol='SoundLevelDecibels')

In [24]:
indexer = StringIndexer(inputCol="SoundLevelDecibels", outputCol="OriginIndex")

In [29]:
#Build the pipeline
pipeline = Pipeline(stages=[indexer,assembler, scaler, lr])

In [30]:
#Split the data
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

In [31]:
# Fit the pipeline
pipelineModel = pipeline.fit(trainingData)

In [32]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  StringIndexer
Pipeline Stage 2 =  VectorAssembler
Pipeline Stage 3 =  StandardScaler
Label column =  SoundLevelDecibels


In [33]:
#Model Evaluation
predictions = pipelineModel.transform(testingData)

In [34]:
#Print the MSE
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

26.547345476448502


In [35]:
#Print the MAE
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

4.012824603136193


In [36]:
#Print the R-Squared(R2)
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

0.4304269886899187


In [37]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  26.55
Mean Absolute Error =  4.01
R Squared =  0.43
Intercept =  133.21


In [38]:
#Model persistance
pipelineModel.write().overwrite().save("Final_Project")